In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [3]:
spark = (
    SparkSession.builder.appName("preprocessing of taxi data")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "15g")
    .getOrCreate()
)

22/09/20 12:34:13 WARN Utils: Your hostname, Lis-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.58.19 instead (on interface en0)
22/09/20 12:34:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/20 12:34:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/20 12:34:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
transaction_20210828_20220227_sdf = spark.read.parquet("../data/curated/transactions_20210828_20220227")

In [6]:
transaction_20210828_20220227_sdf

user_id,merchant_abn,dollar_value,order_id,order_datetime,merchant_name,prod_desc,revenue_level,take_rate,consumer_name,address,state,postcode,gender,consumer_id
7,71041015148,226.70111326425848,ba1c3c0b-6143-49c...,2021-11-26,Orci Ltd,digital goods: bo...,c,2.1,Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685
7,14492521225,28.69248621050033,c82c2951-6a2c-4bf...,2021-11-26,Praesent Eu Assoc...,tent and awning s...,c,2.5,Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685
7,68004106739,93.96901941736229,40d71489-2203-485...,2021-11-26,Nec Ante Ltd,"cable, satellite,...",a,5.6,Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685
7,34096466752,92.43454448910457,293dc228-8398-49f...,2021-11-29,Nullam Enim Ltd,"computers, comput...",b,3.2,Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685
7,62773208456,24.606053328805864,918d63ec-d125-43b...,2021-11-29,Ac Institute,"watch, clock, and...",c,2.6,Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685
7,67874735704,871.400942067755,0dc0a909-7b11-423...,2021-11-29,Ultricies Adipisc...,florists supplies...,c,2.0,Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685
7,36125151647,22.37653874205458,ebd96051-41c5-44d...,2021-11-29,Sed Nec Corp.,"hobby, toy and ga...",c,1.8,Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685
7,86578477987,14.932370248640328,b13248d9-3c6d-4b0...,2021-11-29,Leo In Consulting,"watch, clock, and...",a,6.4,Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685
7,48534649627,16.362338872391526,96a5d5f4-2f24-4c9...,2021-11-29,Dignissim Maecena...,"opticians, optica...",a,6.6,Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685
7,76767266140,144.7959046161148,734de9e3-dcc1-4da...,2021-11-29,Phasellus At Limited,"furniture, home f...",b,4.6,Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685


In [9]:
data = transaction_20210828_20220227_sdf.withColumn("Year", F.year("order_datetime"))\
        .withColumn("Month", F.month("order_datetime"))

In [11]:
transact_agg_by_month = data.groupBy("Year", "Month", "merchant_abn")\
        .agg(F.sum("dollar_value")\
        .alias("sum_transactions"), \
         F.count("order_id").alias("number_of_customers"), \
         F.countDistinct("user_id").alias("distinct_customers"))

In [32]:
transact_agg_by_month

Year,Month,merchant_abn,sum_transactions,number_of_customers,distinct_customers
2021,9,20569508100,10099.236167068768,131,131
2021,12,19933438190,65121.04160676313,4671,4245
2021,10,46970913786,39502.63705626766,202,197
2021,10,71247553576,79348.37054465171,238,236
2021,12,96965554802,13385.328780379374,110,110
2021,11,67915848101,56376.517798328365,156,156
2021,11,90543168331,636310.2107110615,2444,2337
2022,1,22033359776,356058.72007584944,926,913
2022,1,99976658299,134429.57385066163,882,868
2021,9,72553304202,21054.74136332652,329,328


In [33]:
transact_agg_by_monthly_average = transact_agg_by_month.groupby("merchant_abn")\
    .agg(F.count("Month").alias("Number_of_Month"),\
    F.sum("sum_transactions").alias("total_trans"))

In [34]:
final_df = transact_agg_by_monthly_average.withColumn('average_monthly_transaction', transact_agg_by_monthly_average.total_trans/transact_agg_by_monthly_average.Number_of_Month)